In [9]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

system_prompt = """
Bạn là chuyên gia về dược liệu Việt Nam.

Nhiệm vụ: Từ bài viết sau, hãy trích xuất tên cây thuốc và các công dụng hoặc bệnh mà nó có thể hỗ trợ.
Kết quả trả về ngắn gọn, không giải thích thêm, không viết mô tả dài. Cần lấy ra được tối đa khả năng chữa bệnh của cây thuốc.
Giữ đúng format sau:
Tên cây thuốc: [công dụng 1; công dụng 2; công dụng 3]

- Nếu không có công dụng rõ ràng, bỏ qua cây đó.
- Một bài viết cho một loại cây thuốc/ một thuốc, để trên tiêu đề (H1), bỏ qua các bài thuốc, cây thuốc khác nếu có đề cập trong bài.

Ví dụ:
Yến Sào: [phục hồi hệ hô hấp; tăng cường phổi; chậm lão hóa]
"""

response = None
client = None

def extract_info_from_file(client: genai.Client, input_file: str, prompt: str, model: str, temperature: float) -> str:
    """
    Gửi một file markdown đến API của Gemini để trích xuất thông tin thuốc.

    Args:
        client: Đối tượng genai.Client đã được khởi tạo.
        input_file: Đường dẫn đến file markdown đầu vào.
        prompt: System prompt hướng dẫn cho mô hình AI.
        model: Tên mô hình AI để sử dụng (ví dụ: "gemini-1.5-flash").
        temperature: Nhiệt độ cho mô hình (độ sáng tạo).

    Returns:
        Chuỗi thông tin được trích xuất bởi AI, hoặc chuỗi rỗng nếu có lỗi.
    """
    try:
        # 1. Tải file lên dịch vụ của Google
        uploaded_file = client.files.upload(file=input_file)

        # 2. Gửi yêu cầu đến mô hình AI với file đã tải lên
        response = client.models.generate_content(
            model= model,
            contents=[uploaded_file],
            config=types.GenerateContentConfig(
                thinking_config = types.ThinkingConfig(
                    thinking_budget=0,
                ),
                system_instruction=prompt,
                temperature=temperature
                ),
        )

        return response.text.strip()

    except Exception as e:
        print(f"LỖI khi xử lý file '{input_file}': {e}")
        return "" # Trả về chuỗi rỗng nếu có lỗi



def process_drug_files(file_list: list, output_file: str, api_key: str, prompt: str, model: str, temperature: float):
    """
    Xử lý một danh sách các file cây thuốc và ghi kết quả vào một file output.

    Args:
        file_list: Danh sách các đường dẫn file cần xử lý.
        output_file: Đường dẫn đến file output để lưu kết quả.
        api_key: API key của Google AI.
        prompt: System prompt cho AI.
        model: Tên mô hình AI.
        temperature: Nhiệt độ cho AI.
    """
    print(f"Bắt đầu xử lý {len(file_list)} file. Kết quả sẽ được lưu vào '{output_file}'")

    # Khởi tạo client một lần duy nhất
    client = genai.Client(api_key=api_key)

    final_results = []

    # Sử dụng tqdm để hiển thị thanh tiến trình
    for file_path in file_list:
        if not os.path.exists(file_path):
            print(f"CẢNH BÁO: File không tồn tại, bỏ qua: '{file_path}'")
            continue

        # Gọi hàm để trích xuất thông tin từ một file
        raw_result = extract_info_from_file(client, file_path, prompt, model, temperature)
        print(raw_result)
        # Nếu có kết quả trả về, định dạng và thêm vào danh sách
        if raw_result:
            formatted_line = f"- {raw_result} ({file_path})"
            final_results.append(formatted_line)

    # Ghi tất cả kết quả vào file output
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(final_results))
        print(f"\nHoàn tất! Đã lưu kết quả vào file '{output_file}'")
    except IOError as e:
        print(f"\nLỖI: Không thể ghi vào file '{output_file}': {e}")




_api_key = os.environ.get("GOOGLE_API_KEY") # oanhdao2018-test-caythuoc
_temperature = 0
_model =  "gemini-2.5-flash"
_prompt = system_prompt

_prompt = system_prompt
_output_file = "caythuoc_dictionary.txt"

# 2. Danh sách các file đầu vào
# Bạn có thể điền thủ công hoặc dùng code để tự động tìm file
input_files = [
"trungtamthuoc.com/a-giao.md",
"trungtamthuoc.com/a-ke.md",
"trungtamthuoc.com/acai.md",
"trungtamthuoc.com/actiso.md",
"trungtamthuoc.com/am-dia-quyet.md",
"trungtamthuoc.com/an-xoa.md",
"trungtamthuoc.com/ao-coc-sarg.md",
"trungtamthuoc.com/ba-chac.md",
"trungtamthuoc.com/ba-che.md",
"trungtamthuoc.com/ba-dau.md",
"trungtamthuoc.com/ba-dot.md",
"trungtamthuoc.com/ba-gac-bon-la.md",
"trungtamthuoc.com/ba-gac-hoa-do-benth-ex-kurz.md",
"trungtamthuoc.com/ba-gac-la-to.md",
"trungtamthuoc.com/ba-gac-la-vong.md",
"trungtamthuoc.com/ba-gac-phu-tho.md",
"trungtamthuoc.com/ba-kich.md",
"trungtamthuoc.com/ba-soi.md",
"trungtamthuoc.com/ba-thuoc.md",
"trungtamthuoc.com/ba-tu-nhan.md",
"trungtamthuoc.com/bac-ha-cay.md",
"trungtamthuoc.com/bac-ha-dang.md",
"trungtamthuoc.com/bac-ha-nui-miq.md",
"trungtamthuoc.com/bac-ha.md",
"trungtamthuoc.com/bac-thau.md",
"trungtamthuoc.com/bach-anh.md",
"trungtamthuoc.com/bach-benh.md",
"trungtamthuoc.com/bach-bien-dau.md",
"trungtamthuoc.com/bach-bo.md",
"trungtamthuoc.com/bach-cap.md",
"trungtamthuoc.com/bach-chi-nam-kurz.md",
"trungtamthuoc.com/bach-chi.md",
"trungtamthuoc.com/bach-co-dinh-lam.md",
"trungtamthuoc.com/bach-cuong-tam.md",
"trungtamthuoc.com/bach-dan.md",
"trungtamthuoc.com/bach-dau-khau.md",
"trungtamthuoc.com/bach-dong-nu.md",
"trungtamthuoc.com/bach-duong.md",
"trungtamthuoc.com/bach-gioi-tu.md",
"trungtamthuoc.com/bach-hoa-xa-thiet-thao.md",
"trungtamthuoc.com/bach-hoa-xa.md",
"trungtamthuoc.com/bach-hop.md",
"trungtamthuoc.com/bach-liem.md",
"trungtamthuoc.com/bach-linh.md",
"trungtamthuoc.com/bach-phu-tu.md",
"trungtamthuoc.com/bach-qua.md",
"trungtamthuoc.com/bach-tat-le.md",
"trungtamthuoc.com/bach-thao-suong.md",
"trungtamthuoc.com/bach-thuoc.md",
"trungtamthuoc.com/bach-truat.md",
"trungtamthuoc.com/bach-xu.md",
"trungtamthuoc.com/bai-huong.md",
"trungtamthuoc.com/bai-tuong.md",
"trungtamthuoc.com/bam-bam-merr.md",
"trungtamthuoc.com/ban-au.md",
"trungtamthuoc.com/ban-bien-lien.md",
"trungtamthuoc.com/ban-chi-lien.md",
"trungtamthuoc.com/ban-ha-bac-breit.md",
"trungtamthuoc.com/ban-ha-nam.md",
"trungtamthuoc.com/ban-long-sam-ames.md",
"trungtamthuoc.com/ban-tay-ma-sleumer.md",
"trungtamthuoc.com/ban-tron.md",
"trungtamthuoc.com/ban-tu-canh.md",
"trungtamthuoc.com/ban.md",
"trungtamthuoc.com/bang-lang-pers.md",
"trungtamthuoc.com/bang-nuoc.md",
"trungtamthuoc.com/bang.md",
"trungtamthuoc.com/bao-ngu.md",
"trungtamthuoc.com/bao-xuan-hoa-merr.md",
"trungtamthuoc.com/bap-cai.md",
"trungtamthuoc.com/bat-giac-lien.md",
"trungtamthuoc.com/bau-dat-dai.md",
"trungtamthuoc.com/bau-dat-la-mac.md",
"trungtamthuoc.com/bau-dat.md",
"trungtamthuoc.com/bau-giac-tia.md",
"trungtamthuoc.com/bau-nau.md",
"trungtamthuoc.com/bau-ran.md",
"trungtamthuoc.com/bau.md",
"trungtamthuoc.com/beo-cai.md",
"trungtamthuoc.com/beo-hoa-dau.md",
"trungtamthuoc.com/bi-dai.md",
"trungtamthuoc.com/bi-dao.md",
"trungtamthuoc.com/bi-ky-nam.md",
"trungtamthuoc.com/bi-ngo.md",
"trungtamthuoc.com/bien-suc.md",
"trungtamthuoc.com/bim-bim-biec-roth.md",
"trungtamthuoc.com/bim-bim.md",
"trungtamthuoc.com/bim-canh.md",
"trungtamthuoc.com/bim-nap.md",
"trungtamthuoc.com/bim-tia-roth.md",
"trungtamthuoc.com/binh-cu.md",
"trungtamthuoc.com/binh-voi.md",
"trungtamthuoc.com/black-cohosh.md",
"trungtamthuoc.com/blessed-thistle.md",
"trungtamthuoc.com/bo-bo.md",
"trungtamthuoc.com/bo-cap.md",
"trungtamthuoc.com/bo-cau.md",
"trungtamthuoc.com/bo-cho.md",
"trungtamthuoc.com/bo-cong-anh-thap.md",
"trungtamthuoc.com/bo-cong-anh.md",
"trungtamthuoc.com/bo-cu-ve.md",
"trungtamthuoc.com/bo-cui.md",
"trungtamthuoc.com/bo-de.md",
"trungtamthuoc.com/bo-hat-mo.md",
"trungtamthuoc.com/bo-hon.md",
"trungtamthuoc.com/bo-hung.md",
"trungtamthuoc.com/bo-ket-tay-benth.md",
"trungtamthuoc.com/bo-ket.md",
"trungtamthuoc.com/bo-mam-long.md",
"trungtamthuoc.com/bo-mam.md",
"trungtamthuoc.com/bo-may-do.md",
"trungtamthuoc.com/bo-quan-an.md",
"trungtamthuoc.com/bo-quan.md",
"trungtamthuoc.com/bo.md",
"trungtamthuoc.com/boi-loi.md",
"trungtamthuoc.com/boi-ngoi-co.md",
"trungtamthuoc.com/bon-bon-merr.md",
"trungtamthuoc.com/bon-bon.md",
"trungtamthuoc.com/bon-bot.md",
"trungtamthuoc.com/bong-bac.md",
"trungtamthuoc.com/bong-bong-nui.md",
"trungtamthuoc.com/bong-bong-roxb.md",
"trungtamthuoc.com/bong-bong-sw.md",
"trungtamthuoc.com/bong-gung-xanh.md",
"trungtamthuoc.com/bong-nga-truat-mansf.md",
"trungtamthuoc.com/bong-no-thunb.md",
"trungtamthuoc.com/bong-tai.md",
"trungtamthuoc.com/bong-vai-cay.md",
"trungtamthuoc.com/bot-ech-long.md",
"trungtamthuoc.com/bu-oc-leo.md",
"trungtamthuoc.com/bua.md",
"trungtamthuoc.com/bui-tron.md",
"trungtamthuoc.com/bum-bup-gai.md",
"trungtamthuoc.com/bum-bup-truon.md",
"trungtamthuoc.com/bum-sum.md",
"trungtamthuoc.com/bun-thieu.md",
"trungtamthuoc.com/bung-bang-merr.md",
"trungtamthuoc.com/bung-buc.md",
"trungtamthuoc.com/buoi.md",
"trungtamthuoc.com/buom-bac.md",
"trungtamthuoc.com/bup-giam.md",
]

# --- Tùy chọn: Tự động tìm tất cả các file .md trong một thư mục ---
# Bỏ comment đoạn dưới nếu bạn muốn dùng
# import glob
# directory = "trungtamthuoc.com"
# input_files = glob.glob(f"{directory}/*.md")
# -----------------------------------------------------------------

# 3. Gọi hàm xử lý chính
process_drug_files(
    file_list=input_files,
    output_file=_output_file,
    api_key=_api_key,
    prompt=_prompt,
    model=_model,
    temperature=_temperature
)


Bắt đầu xử lý 141 file. Kết quả sẽ được lưu vào 'caythuoc_dictionary.txt'
A Giao: [bồi bổ; cầm máu; an thai; cải thiện sự hấp thụ canxi; tăng khả năng đông máu; tăng nhanh lượng hồng cầu; tăng nhanh sắc tố của máu; chữa chứng loạn dưỡng cơ dần dần; chống choáng; ngăn chặn quá trình lão hóa; tăng cường hoạt động chống oxy hóa; loại bỏ các gốc tự do; điều chỉnh biểu hiện gen liên quan đến lão hóa; cải thiện độ ẩm cho da; làm sáng màu da; chữa bằng huyết; chữa lỵ ra máu; chữa ho ra máu; chữa đại tiểu tiện ra máu; chữa hồi hộp mất ngủ; tư âm; dưỡng huyết; bổ phế; nhuận táo; chữa hư lao sinh ho; chữa phế ung thổ ra mủ; chữa nôn ra máu; chữa ra máu cam; chữa ỉa ra máu; chữa thai sản; chữa bằng lậu âm hư; chữa tâm phiền; chữa mất ngủ; chữa kinh nguyệt ra mãi không ngừng; trị chứng táo bón ở người già; điều trị bệnh lậu; trị khí suy ở tràng vị; trị nóng lạnh thất thường; trị kiết lỵ ra máu; trị kiết lỵ ra mủ; trị tiểu tiện không thông; trị chứng tiểu ra máu trong thời kỳ mang thai; trị chứng s